In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import xarray as xr

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
noch4r = xr.open_dataset('../results/noCH4R.nc')

In [ ]:
ch4r = xr.open_dataset('../results/CH4R.nc')

In [ ]:
noch4r_lifetime = xr.open_dataset('../results/noCH4R_lifetime.nc')

In [ ]:
noch4r.temperature

In [ ]:
ch4r.temperature

In [ ]:
ecs_tcr = pd.read_csv('../results/ecs_tcr.csv', index_col=0)

In [ ]:
temp_norm_19952014 = np.ones(21)
temp_norm_19952014[0] = 0.5
temp_norm_19952014[-1] = 0.5

In [ ]:
colors = {
    'ssp119': '#00a9cf',
    'COFFEE1.1 EN_NPi2020_400f_lowBECCS': '#B8BDAA',
    'ssp534': '#92397a'
}

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
pl.scatter(
    -np.sum(
        ch4r.emissions.loc[
            dict(scenario='ssp119')
        ] - noch4r.emissions.loc[
            dict(scenario='ssp119', specie='CH4')
        ], axis=0
    )/1000, 
    np.max(
        noch4r.temperature.loc[
            dict(scenario='ssp119', layer=0)
        ] - np.average(
            noch4r.temperature.loc[
                dict(scenario='ssp119', layer=0, timebound=np.arange(1995,2016))
            ], axis=0, weights=temp_norm_19952014
        ) + 0.85, axis=0
    ),
    label='ssp119',
    s=5,
    lw=0,
    alpha=0.8,
    color='#00a9cf'
)

pl.scatter(
    -np.sum(
        ch4r.emissions.loc[
            dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS')
        ] - noch4r.emissions.loc[
            dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', specie='CH4')
        ], axis=0
    )/1000, 
    np.max(
        noch4r.temperature.loc[
            dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', layer=0)
        ] - np.average(
            noch4r.temperature.loc[
                dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', layer=0, timebound=np.arange(1995,2016))
            ], axis=0, weights=temp_norm_19952014
        ) + 0.85, axis=0
    ),
    label='IMP-Neg (C2)',
    s=5,
    lw=0,
    alpha=0.8,
    color='#B8BDAA'
)

pl.scatter(
    -np.sum(
        ch4r.emissions.loc[
            dict(scenario='ssp534')
        ] - noch4r.emissions.loc[
            dict(scenario='ssp534', specie='CH4')
        ], axis=0
    )/1000, 
    np.max(
        noch4r.temperature.loc[
            dict(scenario='ssp534', layer=0)
        ] - np.average(
            noch4r.temperature.loc[
                dict(scenario='ssp534', layer=0, timebound=np.arange(1995,2016))
            ], axis=0, weights=temp_norm_19952014
        ) + 0.85, axis=0
    ),
    label='ssp534',
    s=5,
    lw=0,
    alpha=0.8,
    color='#92397a'
)

pl.legend()

pl.xlabel('Cumulative methane removal required 2020-2300 (GtCH$_4$)')
pl.ylabel('Peak warming in underlying ensemble member (K)')
fig.tight_layout()

In [ ]:
scenarios = ['ssp119', 'COFFEE1.1 EN_NPi2020_400f_lowBECCS', 'ssp534']
labels = {
    'ssp119': 'SSP1-1.9',
    'COFFEE1.1 EN_NPi2020_400f_lowBECCS': 'IMP-Neg (C2)',
    'ssp534': 'SSP5-3.4'
}

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        lw=0.5
    );
    ax.plot(
        ch4r.timepoint, 
        np.median(ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(-400, 600)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('Mt CH$_4$ yr$^{-1}$')
ax.legend()
ax.set_title('Ensemble median net methane emissions');

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        ch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(0, 900)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('Mt CH$_4$ yr$^{-1}$')
ax.legend()
ax.set_title('Ensemble median gross methane removal');

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.temperature.loc[dict(scenario=scenario, layer=0)], axis=1), color=colors[scenario],
        lw=0.5
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.temperature.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(0.7, 2.5)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('°C relative to 1850-1900')
ax.axhline(1.5, lw=0.5, ls=':', color='k')
ax.legend()
ax.set_title('Ensemble median global mean surface temperature');

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(
            noch4r.temperature.loc[dict(scenario=scenario, layer=0)] - ch4r.temperature.loc[dict(scenario=scenario)],
            axis=1
        ),
        color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(0, 0.8)
ax.set_ylabel('°C')
ax.axhline(1.5, lw=0.5, ls=':', color='k')
ax.legend()
ax.set_title('Ensemble median avoided warming');

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.concentration.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        lw=0.5
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.concentration.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(0, 2300)
ax.set_ylabel('ppb')
ax.legend()
ax.set_title('Ensemble median methane concentrations');

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        lw=0.5
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_ch4.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(-1, 0.8)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('W m$^{-2}$')
ax.legend()
ax.set_title('Ensemble median methane radiative forcing');

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='Ozone')], axis=1), color=colors[scenario],
        lw=0.5
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_o3.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(-0.1, 0.6)
ax.set_ylabel('W m$^{-2}$')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend()
ax.set_title('Ensemble median ozone radiative forcing');

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing_sum.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        lw=0.5
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_sum.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(1, 5)
ax.set_ylabel('W m$^{-2}$')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend()
ax.set_title('Ensemble median radiative forcing');

In [ ]:
noch4r_lifetime.loc[dict(scenario=scenario)]

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r_lifetime.loc[dict(scenario=scenario)].__xarray_dataarray_variable__, axis=1), color=colors[scenario],
        lw=0.5
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.lifetime.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2000, 2300)
ax.set_ylim(6, 11)
ax.set_ylabel('yr')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend()
ax.set_title('Ensemble median methane lifetime');

In [ ]:
# fig, ax = pl.subplots(2, 2, figsize=(18/2.54, 18/2.54))
# ax[0,0].fill_between(
#     ch4r.timepoint,
#     np.percentile(
#         ch4r.emissions.loc[dict(scenario='ssp119')],
#         5,
#         axis=1
#     ),
#     np.percentile(
#         ch4r.emissions.loc[dict(scenario='ssp119')],
#         95,
#         axis=1
#     ),
#     color='k',
#     alpha=0.3
# )
# ax[0,0].plot(ch4r.timepoint, np.median(ch4r.emissions.loc[dict(scenario='ssp119')], axis=1), color='k');
# ax[0,0].plot(noch4r.timepoint, np.median(noch4r.emissions.loc[dict(scenario='ssp119', specie='CH4')], axis=1), color='r');
# ax[0,0].axhline(0, color='k', lw=0.5, ls=':')
# ax[0,0].set_xlim(2000, 2300)
# ax[0,0].set_ylim(-500, 450)
# ax[0,0].set_title("Net CH$_4$ emissions")
# ax[0,0].set_ylabel("MtCH$_4$ yr$^{-1}$")

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
pl.scatter(-np.sum(emissions_opt - ssp119.emissions[:, 0, :, 3].values, axis=0)/1000, ecs_tcr["TCR"])
pl.xlabel('Cumulative methane removal required (GtCH$_4$)')
pl.ylabel('Transient climate response (K)')
fig.tight_layout()

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
pl.scatter(-np.sum(emissions_opt - ssp119.emissions[:, 0, :, 3].values, axis=0)/1000, ecs_tcr["ECS"])
pl.xlabel('Cumulative methane removal required (GtCH$_4$)')
pl.ylabel('Equilibrium climate sensitivity (K)')
fig.tight_layout()

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
pl.scatter(-np.sum(emissions_opt - ssp119.emissions[:, 0, :, 3].values, axis=0)/1000, ssp119.forcing[255:265, 0, :, 56:58].sum(axis=2).mean(axis=0))
pl.xlabel('Cumulative methane removal required (GtCH$_4$)')
pl.ylabel('2014 aerosol forcing')
fig.tight_layout()

In [ ]:
zec = pd.read_csv('../results/zec_esm-bell.csv', index_col=0)
zec

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
pl.scatter(-np.sum(emissions_opt - ssp119.emissions[:, 0, :, 3].values, axis=0)/1000, zec.zec100_750)
pl.xlabel('Cumulative methane removal required (GtCH$_4$)')
pl.ylabel('ZEC100 (750 PgC bell experiment)')
fig.tight_layout()